# Predicting House Prices in California with `LinearRegression()`

In this lab you will start inspect, analyze, visualize house price data from different districts in California, US. After having performed analysis, EDA and some feature engineering, you will build your own `LinearRegression()`  with `SkLearn`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Part 1 - Inspection and Cleaning


#### Import and Inspect your data

Read the `housing.csv` file and make use of some methods to understand your data better. Below is an explanation of the features you are going to work with:

1. **longitude:**  geographical coordinate, east to west position of district
2. **latitude:**  geographical coordinate, north to south position of district
3. **housing_median_age:** the median age of houses in district
4. **total_rooms** Sum of all rooms in district
5. **total_bedrooms** Sum of all bedrooms in district
6. **population:** total population in district
7. **households:** total households in district
8. **median_income:** median household income in district
9. **median_house_value:** median house value in district
10. **ocean_proximity:** District´s proximity to the ocean

In [ ]:
housing_data = pd.read_csv('housing.csv')

In [ ]:
housing_data

#### Histograms
Make histograms of all your numeric columns in order to get a good understanding of the distribution of your data points. What do you see?

In [ ]:
#selecting num columns
num_cols = housing_data.select_dtypes(include=[np.number]).columns.tolist()

#hist plots
housing_data[num_cols].hist(bins=30, figsize=(15, 10))
plt.suptitle("Histograms of Numeric Columns", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

#### Let's create some features a tidy up our data

1. Locate your NaN values and make a decision on how to handle them. Drop, fill with mean, or something else, it is entirely up to you.

In [ ]:
# 1. Locate NaN values
print("Missing values per column before handling:")
print(housing_data.isnull().sum())

In [ ]:
# 2. Handle missing values
# Numeric columns: fill NaNs with median
numeric_cols = housing_data.select_dtypes(include='number').columns
for col in numeric_cols:
    if housing_data[col].isnull().any():
        housing_data[col].fillna(housing_data[col].median(), inplace=True)

In [ ]:
# Categorical columns: fill NaNs with mode
categorical_cols = housing_data.select_dtypes(exclude='number').columns
for col in categorical_cols:
    if housing_data[col].isnull().any():
        housing_data[col].fillna(housing_data[col].mode()[0], inplace=True)

In [ ]:

# 3. Confirm no more NaNs
print("\nMissing values per column after handling:")
print(housing_data.isnull().sum())

2. Create three new columns by using simple arithmetic operations. Create one column with "rooms per household", one with "population per household",  and one with "bedrooms per room".

3. If you check the largest and smallest values of your "rooms per houshold column" you will see two outliers and two values that are just wrong. Drop the four values by index.

In [ ]:
housing_data['rooms_per_household'] = housing_data['total_rooms'] / housing_data['households']

print("Two largest rooms_per_household values:")
print(housing_data['rooms_per_household'].nlargest(2))

print("Two smallest rooms_per_household values:")
print(housing_data['rooms_per_household'].nsmallest(2))


In [ ]:
idx = list(housing_data['rooms_per_household'].nlargest(2).index) + \
      list(housing_data['rooms_per_household'].nsmallest(2).index)

housing_data_clean = housing_data.drop(index=idx)
print(f"Dropped indices {idx}. New shape: {housing_data_clean.shape}")


# Part 2 - Exploratory Data Analysis



#### Let's find out what factors have an influence on our predicting variable

1. Let's check out the distribution of our "median house value". Visualize your results with 100 bins.

In [ ]:
# Plot the distribution of median_house_value with 100 bins
plt.figure()
plt.hist(housing_data['median_house_value'], bins=100)
plt.title("Distribution of Median House Value")
plt.xlabel("Median House Value")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

2. Check out what variables correlates the most with "median house value"

3. Let's check out the distribution of the column that has the highest correlation to "median house value". Visualize your results with 100 bins.

In [ ]:
plt.figure()
plt.hist(housing_data['median_income'], bins=100)
plt.title("Distribution of Median Income")
plt.xlabel("Median Income")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

4. Visualize the "median house value" and "median income" in a jointplot (kind="reg"). What do you see?

In [ ]:
# Define variables
x = housing_data['median_income']
y = housing_data['median_house_value']

# Fit linear regression
m, b = np.polyfit(x, y, 1)

# Plot scatter + regression line
plt.figure()
plt.scatter(x, y, s=10)
plt.plot(x, m * x + b)
plt.title("Median House Value vs. Median Income (with Regression Line)")
plt.xlabel("Median Income")
plt.ylabel("Median House Value")
plt.tight_layout()
plt.show()

5. Make the same visualization as in the above, but, cahnge the kind parameter to "kde". What extra information does this type of visualization convey, that the one in the above does not?

In [ ]:
# Create a joint KDE plot
sns.jointplot(data=housing_data, x='median_income', y='median_house_value', kind='kde')

plt.tight_layout()
plt.show()

#### Let's get schwifty with some EDA

1. Create a new categorical column from the "median income" with the following quartiles `[0, 0.25, 0.5, 0.75, 0.95, 1]` and label them like this `["Low", "Below_Average", "Above_Average", "High", "Very High"]` and name the column "income_cat"

In [ ]:
# Create income categories based on specified quantile bins
housing_data['income_cat'] = pd.qcut(
    housing_data['median_income'],
    q=[0, 0.25, 0.5, 0.75, 0.95, 1],
    labels=["Low", "Below_Average", "Above_Average", "High", "Very High"]
)

# Display the first few rows to confirm the new column
print(housing_data[['median_income', 'income_cat']].head(10))

# Show counts per category
print("\nValue counts for income_cat:")
print(housing_data['income_cat'].value_counts())

2. Using the Seaborn library, plot the count of your new column and set the `hue` to "ocean_proximity". What interesting things can you see?

In [ ]:


# Plot count of income categories with hue by ocean proximity
plt.figure(figsize=(10, 6))
sns.countplot(
    data=housing_data,
    x='income_cat',
    hue='ocean_proximity'
)
plt.title("Count of Income Categories by Ocean Proximity")
plt.xlabel("Income Category")
plt.ylabel("Count")
plt.legend(title="Ocean Proximity", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


3. Create two barplots where you set "y="median_house_value" on both, and the x is first "income cat" and then "ocean_proximity". How does these two graphs complement what you saw in the graph in your previous question?

In [ ]:
# First barplot: average median_house_value by income_cat
plt.figure(figsize=(10, 5))
sns.barplot(data=housing_data, x='income_cat', y='median_house_value', ci=None)
plt.title('Average Median House Value by Income Category')
plt.xlabel('Income Category')
plt.ylabel('Average Median House Value')
plt.tight_layout()
plt.show()



In [ ]:
# Second barplot: average median_house_value by ocean_proximity
plt.figure(figsize=(10, 5))
sns.barplot(data=housing_data, x='ocean_proximity', y='median_house_value', ci=None)
plt.title('Average Median House Value by Ocean Proximity')
plt.xlabel('Ocean Proximity')
plt.ylabel('Average Median House Value')
plt.tight_layout()
plt.show()

These two barplots show average house values along two dimensions:

By Income Category:

A clear upward trend from Low ($125k) to Very High ($440k).

This confirms our earlier finding that income is strongly tied to house values—each income bucket has a noticeably different mean price.

By Ocean Proximity:

Island tops all others at ~$380k (though this category is very small).

Next highest is Near Ocean ($250k) and Near Bay ($260k).

<1H Ocean is slightly lower (~$240k).

Inland is lowest (~$125k).

This complements the countplot by showing not just how many properties fall into each proximity–and income–bucket, but how their prices differ on average.

4. Create a pivoted dataframe where you have the values of the "income cat" column as indices and the values of the "ocean_proximity" column as columns. Also drop the "ISLAND" column that you'll get.

In [ ]:
# Create pivoted dataframe of counts
pivot_df = housing_data.pivot_table(
    index='income_cat',
    columns='ocean_proximity',
    aggfunc='size',
    fill_value=0
)

# Drop the 'ISLAND' column
pivot_df = pivot_df.drop(columns='ISLAND')



5. Turn your pivoted dataframe into a heatmap. The heatmap should have annotations in integer format.

In [ ]:
# Recreate income_cat column
housing_data['income_cat'] = pd.qcut(
    housing_data['median_income'],
    q=[0, 0.25, 0.5, 0.75, 0.95, 1],
    labels=["Low", "Below_Average", "Above_Average", "High", "Very High"]
)

# Create pivoted DataFrame
pivot_df = housing_data.pivot_table(
    index='income_cat',
    columns='ocean_proximity',
    aggfunc='size',
    fill_value=0
).drop(columns='ISLAND')

# Prepare data for heatmap
data = pivot_df.values
rows = pivot_df.index.tolist()
cols = pivot_df.columns.tolist()

# Plot heatmap
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(data, aspect='auto')

# Set tick labels
ax.set_xticks(np.arange(len(cols)))
ax.set_xticklabels(cols)
ax.set_yticks(np.arange(len(rows)))
ax.set_yticklabels(rows)

# Annotate each cell with integer count
for i in range(len(rows)):
    for j in range(len(cols)):
        ax.text(j, i, int(data[i, j]), ha='center', va='center')

# Labels and title
ax.set_xlabel("Ocean Proximity")
ax.set_ylabel("Income Category")
ax.set_title("Counts by Income Category and Ocean Proximity")
fig.colorbar(im, ax=ax, orientation='vertical', label='Count')

plt.tight_layout()
plt.show()

# Part 3 - Preparing your Data



#### Splitting, Preparing and Engineering some Features

1. Let's drop the "income_cat" column as it has served its purpose already. We don't need for our model as we already have "median income".
Not dropping "incom cat" will lead to multicolinearity.

In [ ]:
# Confirm drop
print("Columns after dropping income_cat:")
print(housing_data.columns.tolist())



2. Select your floating point columns and standardize your data by calculating the Z-score. You can apply the `stats.zscore()` method in a lambda function. Save your results to a variable called `z_scored`.

In [ ]:
import scipy.stats as stats

In [ ]:
# 1. Select floating-point columns
float_cols = housing_data.select_dtypes(include=['float64']).columns

# 2. Standardize using Z-score
z_scored = housing_data[float_cols].apply(lambda col: stats.zscore(col))

# 3. Preview the standardized data
print("Floating-point columns standardized (first 5 rows):")
print(z_scored.head())


3. Turn the only categorical columns into dummies. Be vary of the dummy trap, to avoid multicolinearity.

In [ ]:
# Identify categorical columns
cat_cols = housing_data.select_dtypes(include=['object', 'category']).columns.tolist()

# Create dummies while avoiding the dummy trap (drop_first)
housing_cat_dummies = pd.get_dummies(housing_data[cat_cols], drop_first=True)

# Combine with the numeric data
housing_prepared = pd.concat(
    [housing_data.drop(columns=cat_cols), housing_cat_dummies],
    axis=1
)

# Preview the prepared DataFrame
print("Categorical columns converted to dummies (first 5 rows):")
print(housing_prepared.head())

print("\nColumns after encoding:")
print(housing_prepared.columns.tolist())

4. Save our predicting variable to `y`.

In [ ]:
y = housing_prepared['median_house_value']

# Quick check
print("Preview of y (target variable):")
print(y.head())
print(f"\nShape of y: {y.shape}")

5. Concatenate `z_scored` and `dummies` and drop the predicting variable. Save to the varible `X`.

In [ ]:
# 1. Concatenate standardized numeric features and dummy variables
X = pd.concat([z_scored, housing_cat_dummies], axis=1)

# 2. Drop the predicting variable if it ended up in X
if 'median_house_value' in X.columns:
    X = X.drop('median_house_value', axis=1)

# 3. Preview the resulting feature matrix
print("Preview of X (first 5 rows):")
print(X.head())
print(f"\nShape of X: {X.shape}")

# Part 4 - Machine Learning




#### Train, Test, Split

1. Import `train_test_split` and split your data accordingly. Choose an appropriate test size.

In [ ]:

# Assuming X and y are already defined
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preview the split sizes
print("Training set shape (X_train):", X_train.shape)
print("Test set shape (X_test):", X_test.shape)
print("Training target shape (y_train):", y_train.shape)
print("Test target shape (y_test):", y_test.shape)

#### Building and Training our Model

2. Build, fit and train a `LinearRegression` model.

In [ ]:

# Instantiate and train the model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Display coefficients and intercept
coefficients = lin_reg.coef_
intercept = lin_reg.intercept_
train_r2 = lin_reg.score(X_train, y_train)

print("Intercept:", intercept)
print("Coefficients:", coefficients)
print("Training R² score:", train_r2)


3. In a scatterplot, visualize the y_train on your x-axis and your predictions on the y-axis. How does your training predictions look?

In [ ]:


# Generate predictions on the training set
y_train_pred = lin_reg.predict(X_train)

# Scatterplot of actual vs. predicted
plt.figure()
plt.scatter(y_train, y_train_pred, s=10)
# Plot 45-degree line for reference
min_val = min(y_train.min(), y_train_pred.min())
max_val = max(y_train.max(), y_train_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], linewidth=2)
plt.title("Actual vs Predicted Median House Value (Training Set)")
plt.xlabel("Actual Median House Value")
plt.ylabel("Predicted Median House Value")
plt.tight_layout()
plt.show()

4. From the sklearn metrics module, print the mean_squared_error and R^2-score. What does the metrics tell us?

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Generate predictions on the test set
y_test_pred = lin_reg.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_test_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_test_pred)

# Display the results
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

Mean Squared Error (MSE): 4,838,614,541.14

Root MSE (RMSE): 69,560.15

R² Score: 0.6308

#### Final Predictions

1. Now you are ready to make prediction on the test data. Do that and visualize your results in a new scatterplot.

In [ ]:
import matplotlib.pyplot as plt

# Generate predictions on the test set
y_test_pred = lin_reg.predict(X_test)

# Scatterplot of actual vs. predicted for test set
plt.figure()
plt.scatter(y_test, y_test_pred, s=10)
# Plot 45-degree reference line
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], color='orange', linewidth=2)
plt.title("Actual vs Predicted Median House Value (Test Set)")
plt.xlabel("Actual Median House Value")
plt.ylabel("Predicted Median House Value")
plt.tight_layout()
plt.show()


2. Print the mean_squared_error and R^2-score again. What has happened?

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Recompute metrics on the test set
mse_new = mean_squared_error(y_test, y_test_pred)
rmse_new = mse_new ** 0.5
r2_new = r2_score(y_test, y_test_pred)

print(f"Mean Squared Error (MSE): {mse_new:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_new:.2f}")
print(f"R² Score: {r2_new:.4f}")


MSE: 4,838,614,541.14

RMSE: 69,560.15

R²: 0.6308

3. There is another metric called Root mean squared error, Which is the square root of the MSE. Calculate the RMSE.

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE directly with squared=False
rmse = mean_squared_error(y_test, y_test_pred)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


# Bonus Questions 1

1. Create a dataframe with two columns, one consisting of the y_test and one of your model's predictions.

2. Make a series of of your new dataframe, by calculating the predicted error in absolut numbers. Save this series to variable name `absolute_errors`.

3. If you take the mean of your series, you will get the mean absolute errors, which is another metric for Linear Regressions.

# Bonus Question 2 - Build a Random Forest Regressor

1. Build, fit and train a `RandomForestRegressor` model. Do this by following the same staps that you followed when building your `LinearRegression`.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Instantiate the RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=42, n_jobs=-1)

# Fit the model on the training data
rf_reg.fit(X_train, y_train)

# Generate predictions
y_train_pred_rf = rf_reg.predict(X_train)
y_test_pred_rf = rf_reg.predict(X_test)

# Calculate metrics
train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf = r2_score(y_test, y_test_pred_rf)
mse_rf = mean_squared_error(y_test, y_test_pred_rf)
rmse_rf = mse_rf ** 0.5

# Display results
print(f"Random Forest Training R²: {train_r2_rf:.4f}")
print(f"Random Forest Test R²: {test_r2_rf:.4f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")
